In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline 
from sklearn.metrics import accuracy_score
from scipy.special import expit
from ipywidgets import widgets as wd

df_imputed = pd.read_csv('responses.csv', sep=",")

In [115]:
df_imputed = df_imputed.dropna()

In [116]:
for col in ['Smoking', 'Alcohol', 'Punctuality', 'Lying', 'Internet usage',
        'Gender', 'Left - right handed', 'Education', 'Only child',
        'Village - town', 'House - block of flats']:
    df_imputed = df_imputed.drop(col,1)

In [117]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'Fear of public speaking' in df_imputed:
    y = df_imputed['Fear of public speaking'].values # get the labels we want
    del df_imputed['Fear of public speaking'] # get rid of the class label
    X = df_imputed.values # use everything else to predict!

num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(
                         n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [118]:
%%time
# from last time, our logistic regression algorithm is given by (including everything we previously had):
class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += 2 * self.w_[1:] * self.C
        
        return gradient
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

# blr = BinaryLogisticRegression(eta=0.1,iterations=500,C=0.001)

# blr.fit(X,y)
# print(blr)

# yhat = blr.predict(X)
# print('Accuracy of: ',accuracy_score(y,yhat+1))

CPU times: user 33 µs, sys: 0 ns, total: 33 µs
Wall time: 37 µs


In [119]:
%%time
from numpy.linalg import pinv
class HessianBinaryLogisticRegression(BinaryLogisticRegression):
    # just overwrite gradient function
    def _get_gradient(self,X,y):
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X + 2 * self.C # calculate the hessian

        ydiff = y-g # get y difference
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += 2 * self.w_[1:] * self.C
        
        return pinv(hessian) @ gradient
       
# hlr = HessianBinaryLogisticRegression(eta=0.1,iterations=20,C=0.1) # note that we need only a few iterations here

# hlr.fit(X,y)
# yhat = hlr.predict(X)
# print(hlr)
# print('Accuracy of: ',accuracy_score(y,yhat+1))

CPU times: user 36 µs, sys: 1e+03 ns, total: 37 µs
Wall time: 40.1 µs


In [120]:
%%time
# and we can update this to use a line search along the gradient like this:
from scipy.optimize import minimize_scalar
import copy
class LineSearchLogisticRegression(BinaryLogisticRegression):
    
    # define custom line search for problem
    @staticmethod
    def line_search_function(eta,X,y,w,grad):
        wnew = w + grad*eta
        yhat = (1/(1+np.exp(-X @ wnew)))>0.5
        return np.sum((y-yhat)**2)+np.sum(wnew**2)
    @staticmethod
    def line_search_function_l1(eta,X,y,w,grad):
        if(math.sin(w) < 0 ):
            w -=1
        elif(math.sin(w) > 0):
            w += 1
        else:
            w = w
        wnew = w + grad*eta
        yhat = (1/(1+np.exp(-X @ wnew)))>0.5
        return np.sum((y-yhat)**2)+np.sum(math.fabs(wnew))
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            
            # do line search in gradient direction, using scipy function
            opts = {'maxiter':self.iters/20} # unclear exactly what this should be
            res = minimize_scalar(self.line_search_function, # objective function to optimize
                                  bounds=(self.eta/1000,self.eta*10), #bounds to optimize
                                  args=(Xb,y,self.w_,gradient), # additional argument for objective function
                                  method='bounded', # bounded optimization for speed
                                  options=opts) # set max iterations
            
            eta = res.x # get optimal learning rate
            self.w_ += gradient*eta # set new function values
                
            

# lslr = LineSearchLogisticRegression(eta=0.1,iterations=110, C=0.001)

# lslr.fit(X,y)

# yhat = lslr.predict(X)
# print(lslr)
# print('Accuracy of: ',accuracy_score(y,yhat))         

CPU times: user 44 µs, sys: 1e+03 ns, total: 45 µs
Wall time: 47 µs


In [121]:
%%time
class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += 2 * self.w_[1:] * self.C
        
        return gradient
    
    
# slr = StochasticLogisticRegression(0.1,1000, C=0.001) # take a lot more steps!!

# slr.fit(X,y)

# yhat = slr.predict(X)
# print(slr)
# print('Accuracy of: ',accuracy_score(y,yhat))      

CPU times: user 27 µs, sys: 0 ns, total: 27 µs
Wall time: 31 µs


In [122]:
%%time
# for this, we won't perform our own BFGS implementation 
# (it takes a good deal of code and understanding of the algorithm)
# luckily for us, scipy has its own BFGS implementation:
from scipy.optimize import fmin_bfgs
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):
    
    @staticmethod
    def objective_function(w,X,y,C):
        g = expit(X @ w)
        return -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + C*sum(w**2) #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += 2 * w[1:] * C
        return -gradient
    
    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)
        
        self.w_ = self.w_.reshape((num_features,1))
            
# bfgslr = BFGSBinaryLogisticRegression(_,2) # note that we need only a few iterations here

# bfgslr.fit(X,y)
# yhat = bfgslr.predict(X)
# print(bfgslr)
# print('Accuracy of: ',accuracy_score(y,yhat+1))

CPU times: user 39 µs, sys: 1 µs, total: 40 µs
Wall time: 42.2 µs


In [134]:
class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.0001, optimization=None):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.classifiers_ = []
        self.optimization = optimization
        print(self.optimization)
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        print(optimization)
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            #hblr = HessianBinaryLogisticRegression(self.eta,self.iters,self.C)
            if(self.optimization == "BFGSBinaryLogisticRegression"):
                hblr = BFGSBinaryLogisticRegression(self.eta,self.iters,self.C)
            elif(self.optimization == "StochasticLogisticRegression"):
                hblr = StochasticLogisticRegression(self.eta,self.iters,self.C)
            else:
                hblr = LineSearchLogisticRegression(self.eta,self.iters,self.C)

            hblr.fit(X,y_binary)
            #print(accuracy(y_binary,hblr.predict(X)))
            # add the trained classifier to the list
            self.classifiers_.append(hblr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row
    



In [124]:
cost_vals = np.logspace(-3,-2,15)
def lr_explor(cost_idx):
    C = cost_vals[cost_idx]
    lr_clf = MultiClassLogisticRegression(eta=0.1,
                                           iterations=2500,
                                           C=C) # get object
    
    plot_decision_boundaries(lr_clf,X,y,title="C=%.5f"%(C))
wd.interact(lr_explor,cost_idx=(0,15,1),__manual=True)

<function __main__.lr_explor>

In [135]:
# run logistic regression and vary some parameters
from sklearn import metrics as mt

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants


lr_clf = MultiClassLogisticRegression(eta=0.1,iterations=10, C=0.0001, optimization="BFGSBinaryLogisticRegression") # get object

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
#     print(X_train)
#     print(y_train)
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat+1)
    conf = mt.confusion_matrix(y_test,y_hat+1)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1
    
# Also note that every time you run the above code
#   it randomly creates a new training and testing set, 
#   so accuracy will be different each time

BFGSBinaryLogisticRegression
LineSearchLogisticRegression
====Iteration 0  ====
accuracy 0.266666666667
confusion matrix
 [[ 8  7  5  3  0]
 [ 7  9  9  6  0]
 [ 1  9 16  5  3]
 [ 4  9  9  3  5]
 [ 2  1  7  7  0]]
LineSearchLogisticRegression
====Iteration 1  ====
accuracy 0.296296296296
confusion matrix
 [[ 9  6  8  2  0]
 [ 2  7  9  8  1]
 [ 4 10 17 12  4]
 [ 0  2 11  4  4]
 [ 0  0  6  6  3]]
LineSearchLogisticRegression
====Iteration 2  ====
accuracy 0.333333333333
confusion matrix
 [[13  3  4  1  0]
 [13  9 14  1  0]
 [ 7  6 15  9  7]
 [ 1  2  9  5  1]
 [ 0  2  7  3  3]]


/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log
/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log


In [126]:
%%time
lr = MultiClassLogisticRegression(eta=0.1,iterations=10,C=0.0001)
lr.fit(X,y)
print(lr)

yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat+1))

LineSearchLogisticRegression


/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: overflow encountered in exp


MultiClass Logistic Regression Object with coefficients:
[[ -1.63349907e-01  -7.53109219e-01  -4.18890314e-01  -5.37218753e-01
   -3.74173244e-01  -3.18686580e-01  -4.16782071e-01  -4.37504622e-01
   -6.38465654e-01  -5.43033776e-01  -3.47500475e-01  -4.29742118e-01
   -4.66071886e-01  -3.39931617e-01  -4.23360747e-01  -4.12940095e-01
   -4.03825684e-01  -4.10749643e-01  -4.63371730e-01  -3.17367557e-01
   -7.98096680e-01  -3.66735922e-01  -7.03349904e-01  -6.64868032e-01
   -6.03155294e-01  -5.82584381e-01  -3.73393149e-01  -6.83038405e-01
   -7.22700445e-01  -5.59667719e-01  -2.52170908e-01  -4.56243148e-01
   -2.84375526e-01  -3.61969532e-01  -1.24642188e-01  -3.19311378e-01
   -2.45098971e-01  -6.62726906e-01  -3.36484250e-01  -2.02482825e-01
   -6.35239353e-01  -5.05425187e-01  -5.29008655e-01  -4.21314614e-01
   -5.65733032e-01  -5.03379054e-01   2.33386913e-02  -2.61860791e-01
   -3.41004010e-01  -2.84991200e-01  -7.81667360e-01  -3.33455253e-01
   -1.52190045e-01  -2.13482608e-

In [127]:
# linear boundaries visualization from sklearn documentation
from matplotlib import pyplot as plt
import copy
%matplotlib inline

def plot_decision_boundaries(lr,Xin,y,title=''):
    Xb = copy.deepcopy(Xin)
    lr.fit(Xb[:,:2],y) # train only on two features

    h=0.01
    # create a mesh to plot in
    x_min, x_max = Xb[:, 0].min() - 1, Xb[:, 0].max() + 1
    y_min, y_max = Xb[:, 1].min() - 1, Xb[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # get prediction values
    Z = lr.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.5)

    # Plot also the training points
    plt.scatter(Xb[:, 0], Xb[:, 1], c=y, cmap=plt.cm.Paired)
    plt.xlabel('Feature')
    plt.ylabel('Fear of Public Speaking')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(title)
    plt.show()


In [128]:


costs = [n for n in np.arange(0,0.01,0.005)]
optimizations = ["BFGSBinaryLogisticRegression","StochasticLogisticRegression","LineSearchLogisticRegression"]

for optimization in optimizations:
    for cost in costs:
        %%time
        lr = MultiClassLogisticRegression(eta=0.1,
                                           iterations=10,
                                           C=cost,optimization=optimization) # get object
        lr.fit(X,y)
#         print(lr)
        yhat = lr.predict(X)
        print('For ',optimization,' and cost ', cost,' Accuracy of: ',accuracy_score(y,yhat+1))
        


CPU times: user 19 µs, sys: 1e+03 ns, total: 20 µs
Wall time: 6.91 µs
BFGSBinaryLogisticRegression
IN
IN
IN
IN
IN
For  BFGSBinaryLogisticRegression  and cost  0.0  Accuracy of:  0.635014836795
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
BFGSBinaryLogisticRegression
IN
IN
IN
IN
IN


/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log
/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log
/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log
/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log
/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: overflow encountered in exp


For  BFGSBinaryLogisticRegression  and cost  0.005  Accuracy of:  0.63649851632
CPU times: user 13 µs, sys: 1e+03 ns, total: 14 µs
Wall time: 7.87 µs
StochasticLogisticRegression
For  StochasticLogisticRegression  and cost  0.0  Accuracy of:  0.166172106825
CPU times: user 16 µs, sys: 1e+03 ns, total: 17 µs
Wall time: 6.91 µs
StochasticLogisticRegression
For  StochasticLogisticRegression  and cost  0.005  Accuracy of:  0.23293768546
CPU times: user 13 µs, sys: 1 µs, total: 14 µs
Wall time: 7.15 µs
LineSearchLogisticRegression
For  LineSearchLogisticRegression  and cost  0.0  Accuracy of:  0.23293768546
CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 5.01 µs
LineSearchLogisticRegression
For  LineSearchLogisticRegression  and cost  0.005  Accuracy of:  0.23293768546


/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: overflow encountered in exp


In [129]:
%%time
from sklearn.linear_model import LogisticRegression as SKLogisticRegression

lr_sk = SKLogisticRegression(solver='lbfgs')#,max_iter=100,C=0.005) 
lr_sk.fit(X,y)
# print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
# yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

Accuracy of:  0.166172106825
CPU times: user 468 ms, sys: 17 ms, total: 485 ms
Wall time: 127 ms
